In [ ]:
from calendar import monthrange
from datetime import datetime
import pandas as pd
from pandas.api.types import CategoricalDtype
import os
import json

import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go

# Load locale custome modules
import sys
sys.path.append(os.path.abspath("../../tools"))
from data_pipeline_lib import *

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', None)

In [ ]:
root_dir =  os.path.abspath(os.getcwd()+"..\\..\\..\\..\\..\\")
tmp_dir = root_dir + "\\.tmp"
data_dir = root_dir + "\\data\\"

site_metadata_filename = data_dir + "site-metadata.csv"

In [ ]:
# "Golden" Sites
tier1_sites = ["US-MMS", "US-Vcp", "FR-Pue", "CH-Lae", "US-Var", "US-Ne2", "ES-LJu", "US-Ton"]
tier2_sites = ["US-UMB", "US-Me2", "FI-Hyy", "US-NR1", "IT-Lav", "US-Wkg", "US-ARM", "US-SRM"]

target_sites = tier1_sites + tier2_sites
target_sites

['US-MMS',
 'US-Vcp',
 'FR-Pue',
 'CH-Lae',
 'US-Var',
 'US-Ne2',
 'ES-LJu',
 'US-Ton',
 'US-UMB',
 'US-Me2',
 'FI-Hyy',
 'US-NR1',
 'IT-Lav',
 'US-Wkg',
 'US-ARM',
 'US-SRM']

In [ ]:
# Load Site data
site_metadata_df = pd.read_csv(site_metadata_filename, usecols=['site_id','filename'])

# only focus on target sites
site_metadata_df= site_metadata_df.loc[site_metadata_df['site_id'].isin(target_sites)]
print(f"size:{site_metadata_df.shape}")
site_metadata_df

size:(16, 2)


,site_id,filename
67,FR-Pue,data_full_half_hourly_raw_v0_1_FR-Pue.csv
117,US-NR1,data_full_half_hourly_raw_v0_1_US-NR1.csv
119,US-Ne2,NaN
124,US-SRM,data_full_half_hourly_raw_v0_1_US-SRM.csv
127,US-Ton,data_full_half_hourly_raw_v0_1_US-Ton.csv
130,US-Var,data_full_half_hourly_raw_v0_1_US-Var.csv
144,US-Wkg,data_full_half_hourly_raw_v0_1_US-Wkg.csv
166,US-ARM,data_full_half_hourly_raw_v0_1_US-ARM.csv
181,US-MMS,NaN
182,US-Me2,data_full_half_hourly_raw_v0_1_US-Me2.csv


In [ ]:
all_features = ['TIMESTAMP_START', 'TIMESTAMP_END', 'TA_F', 'TA_F_QC', 'TA_ERA',
       'SW_IN_POT', 'SW_IN_F', 'SW_IN_F_QC', 'SW_IN_ERA', 'LW_IN_F',
       'LW_IN_F_QC', 'LW_IN_ERA', 'VPD_F', 'VPD_F_QC', 'VPD_ERA', 'P_F',
       'P_F_QC', 'P_ERA', 'PA_F', 'PA_F_QC', 'PA_ERA', 'NETRAD', 'PPFD_IN',
       'G_F_MDS', 'G_F_MDS_QC', 'LE_F_MDS', 'LE_F_MDS_QC', 'LE_CORR',
       'H_F_MDS', 'H_F_MDS_QC', 'H_CORR', 'NEE_VUT_REF', 'NEE_VUT_REF_QC',
       'NEE_CUT_REF', 'NEE_CUT_REF_QC', 'GPP_NT_VUT_REF', 'GPP_DT_VUT_REF',
       'GPP_NT_CUT_REF', 'GPP_DT_CUT_REF', 'RECO_NT_VUT_REF',
       'RECO_DT_VUT_REF', 'RECO_NT_CUT_REF', 'RECO_DT_CUT_REF', 'datetime',
       'year', 'month', 'day', 'hour', 'SITE_ID', 'date', 'NEE_VUT_REF_qa',
       'SW_DIF', 'EVI', 'NDVI', 'NIRv', 'b1', 'b2', 'b3', 'b4', 'b5', 'b6',
       'b7', 'IGBP', 'koppen']
qc_flag_dtype = CategoricalDtype([0, 1, 2, 3], ordered=True)
qc_flags_features = [s for s in all_features if "_QC" in s]

In [ ]:
na_df = None
for i, r in site_metadata_df.iterrows():
    if not r.filename or type(r.filename) != type(""):
        print(f'\nERROR: {r.site_id} is mssing hourly data.')
        continue

    local_filename = tmp_dir + "\\" + r.filename
    site_df = pd.read_csv(local_filename)
    site_df['datetime'] = pd.to_datetime(site_df['datetime'])
    site_df['date'] = pd.to_datetime(site_df['date'])
    site_df[qc_flags_features] = site_df[qc_flags_features].astype(qc_flag_dtype)
    
    print(f"\n{r.site_id}")
    
    na_count_df = pd.DataFrame(site_df.isna().sum()).T
    na_count_df['site_id'] = r.site_id
    display(na_count_df)
    
    
    if type(na_df) == type(None):
        na_df = na_count_df
    else:
        na_df = pd.concat([na_df, na_count_df])


FR-Pue


,TIMESTAMP_START,TIMESTAMP_END,TA_F,TA_F_QC,TA_ERA,SW_IN_POT,SW_IN_F,SW_IN_F_QC,SW_IN_ERA,LW_IN_F,LW_IN_F_QC,LW_IN_ERA,VPD_F,VPD_F_QC,VPD_ERA,P_F,P_F_QC,P_ERA,PA_F,PA_F_QC,PA_ERA,NETRAD,PPFD_IN,G_F_MDS,G_F_MDS_QC,LE_F_MDS,LE_F_MDS_QC,LE_CORR,H_F_MDS,H_F_MDS_QC,H_CORR,NEE_VUT_REF,NEE_VUT_REF_QC,NEE_CUT_REF,NEE_CUT_REF_QC,GPP_NT_VUT_REF,GPP_DT_VUT_REF,GPP_NT_CUT_REF,GPP_DT_CUT_REF,RECO_NT_VUT_REF,RECO_DT_VUT_REF,RECO_NT_CUT_REF,RECO_DT_CUT_REF,datetime,year,month,day,hour,SITE_ID,date,NEE_VUT_REF_qa,SW_DIF,EVI,NDVI,NIRv,b1,b2,b3,b4,b5,b6,b7,IGBP,koppen,site_id
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4325,22234,102266,102266,0,0,50149,0,0,50149,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,12287,245760,9696,9552,9552,9552,9552,9696,9552,10656,11280,9648,0,0,FR-Pue



US-NR1


,TIMESTAMP_START,TIMESTAMP_END,TA_F,TA_F_QC,TA_ERA,SW_IN_POT,SW_IN_F,SW_IN_F_QC,SW_IN_ERA,LW_IN_F,LW_IN_F_QC,LW_IN_ERA,VPD_F,VPD_F_QC,VPD_ERA,P_F,P_F_QC,P_ERA,PA_F,PA_F_QC,PA_ERA,NETRAD,PPFD_IN,G_F_MDS,G_F_MDS_QC,LE_F_MDS,LE_F_MDS_QC,LE_CORR,H_F_MDS,H_F_MDS_QC,H_CORR,NEE_VUT_REF,NEE_VUT_REF_QC,NEE_CUT_REF,NEE_CUT_REF_QC,GPP_NT_VUT_REF,GPP_DT_VUT_REF,GPP_NT_CUT_REF,GPP_DT_CUT_REF,RECO_NT_VUT_REF,RECO_DT_VUT_REF,RECO_NT_CUT_REF,RECO_DT_CUT_REF,datetime,year,month,day,hour,SITE_ID,date,NEE_VUT_REF_qa,SW_DIF,EVI,NDVI,NIRv,b1,b2,b3,b4,b5,b6,b7,IGBP,koppen,site_id
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1794,1483,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,11177,270768,57984,57792,57792,57792,57792,57984,57888,58416,77424,57792,0,0,US-NR1



ERROR: US-Ne2 is mssing hourly data.

US-SRM


...

,TIMESTAMP_START,TIMESTAMP_END,TA_F,TA_F_QC,TA_ERA,SW_IN_POT,SW_IN_F,SW_IN_F_QC,SW_IN_ERA,LW_IN_F,LW_IN_F_QC,LW_IN_ERA,VPD_F,VPD_F_QC,VPD_ERA,P_F,P_F_QC,P_ERA,PA_F,PA_F_QC,PA_ERA,NETRAD,PPFD_IN,G_F_MDS,G_F_MDS_QC,LE_F_MDS,LE_F_MDS_QC,LE_CORR,H_F_MDS,H_F_MDS_QC,H_CORR,NEE_VUT_REF,NEE_VUT_REF_QC,NEE_CUT_REF,NEE_CUT_REF_QC,GPP_NT_VUT_REF,GPP_DT_VUT_REF,GPP_NT_CUT_REF,GPP_DT_CUT_REF,RECO_NT_VUT_REF,RECO_DT_VUT_REF,RECO_NT_CUT_REF,RECO_DT_CUT_REF,datetime,year,month,day,hour,SITE_ID,date,NEE_VUT_REF_qa,SW_DIF,EVI,NDVI,NIRv,b1,b2,b3,b4,b5,b6,b7,IGBP,koppen,site_id
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,9976,12071,0,0,0,0,0,0,0,0,0,0,0,0,8880,0,0,0,8880,0,0,0,0,0,0,0,0,0,0,9326,239616,4944,4944,4944,4944,4944,4944,4944,4944,7248,4944,0,0,ES-LJu



FI-Hyy


,TIMESTAMP_START,TIMESTAMP_END,TA_F,TA_F_QC,TA_ERA,SW_IN_POT,SW_IN_F,SW_IN_F_QC,SW_IN_ERA,LW_IN_F,LW_IN_F_QC,LW_IN_ERA,VPD_F,VPD_F_QC,VPD_ERA,P_F,P_F_QC,P_ERA,PA_F,PA_F_QC,PA_ERA,NETRAD,PPFD_IN,G_F_MDS,G_F_MDS_QC,LE_F_MDS,LE_F_MDS_QC,LE_CORR,H_F_MDS,H_F_MDS_QC,H_CORR,NEE_VUT_REF,NEE_VUT_REF_QC,NEE_CUT_REF,NEE_CUT_REF_QC,GPP_NT_VUT_REF,GPP_DT_VUT_REF,GPP_NT_CUT_REF,GPP_DT_CUT_REF,RECO_NT_VUT_REF,RECO_DT_VUT_REF,RECO_NT_CUT_REF,RECO_DT_CUT_REF,datetime,year,month,day,hour,SITE_ID,date,NEE_VUT_REF_qa,SW_DIF,EVI,NDVI,NIRv,b1,b2,b3,b4,b5,b6,b7,IGBP,koppen,site_id
0,0,0,0,0,17568,0,0,0,17568,0,0,17568,0,0,17568,17568,17568,17568,17568,17568,17568,118056,8661,138719,138719,0,0,128493,0,0,128493,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,11307,238688,167904,167136,167136,166992,165936,167040,167088,166368,180192,166560,0,0,FI-Hyy



IT-Lav


,TIMESTAMP_START,TIMESTAMP_END,TA_F,TA_F_QC,TA_ERA,SW_IN_POT,SW_IN_F,SW_IN_F_QC,SW_IN_ERA,LW_IN_F,LW_IN_F_QC,LW_IN_ERA,VPD_F,VPD_F_QC,VPD_ERA,P_F,P_F_QC,P_ERA,PA_F,PA_F_QC,PA_ERA,NETRAD,PPFD_IN,G_F_MDS,G_F_MDS_QC,LE_F_MDS,LE_F_MDS_QC,LE_CORR,H_F_MDS,H_F_MDS_QC,H_CORR,NEE_VUT_REF,NEE_VUT_REF_QC,NEE_CUT_REF,NEE_CUT_REF_QC,GPP_NT_VUT_REF,GPP_DT_VUT_REF,GPP_NT_CUT_REF,GPP_DT_CUT_REF,RECO_NT_VUT_REF,RECO_DT_VUT_REF,RECO_NT_CUT_REF,RECO_DT_CUT_REF,datetime,year,month,day,hour,SITE_ID,date,NEE_VUT_REF_qa,SW_DIF,EVI,NDVI,NIRv,b1,b2,b3,b4,b5,b6,b7,IGBP,koppen,site_id
0,0,0,0,0,17568,0,0,0,17568,0,0,17568,0,0,17568,17568,17568,17568,17568,17568,17568,106031,15996,19326,19326,0,0,65110,0,0,65110,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,11841,297840,25056,21744,21744,21744,21600,25056,21696,23472,40272,21600,0,0,IT-Lav


In [ ]:
na_df

,TIMESTAMP_START,TIMESTAMP_END,TA_F,TA_F_QC,TA_ERA,SW_IN_POT,SW_IN_F,SW_IN_F_QC,SW_IN_ERA,LW_IN_F,LW_IN_F_QC,LW_IN_ERA,VPD_F,VPD_F_QC,VPD_ERA,P_F,P_F_QC,P_ERA,PA_F,PA_F_QC,PA_ERA,NETRAD,PPFD_IN,G_F_MDS,G_F_MDS_QC,LE_F_MDS,LE_F_MDS_QC,LE_CORR,H_F_MDS,H_F_MDS_QC,H_CORR,NEE_VUT_REF,NEE_VUT_REF_QC,NEE_CUT_REF,NEE_CUT_REF_QC,GPP_NT_VUT_REF,GPP_DT_VUT_REF,GPP_NT_CUT_REF,GPP_DT_CUT_REF,RECO_NT_VUT_REF,RECO_DT_VUT_REF,RECO_NT_CUT_REF,RECO_DT_CUT_REF,datetime,year,month,day,hour,SITE_ID,date,NEE_VUT_REF_qa,SW_DIF,EVI,NDVI,NIRv,b1,b2,b3,b4,b5,b6,b7,IGBP,koppen,site_id
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4325,22234,102266,102266,0,0,50149,0,0,50149,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,12287,245760,9696,9552,9552,9552,9552,9696,9552,10656,11280,9648,0,0,FR-Pue
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1794,1483,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,11177,270768,57984,57792,57792,57792,57792,57984,57888,58416,77424,57792,0,0,US-NR1
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2643,757,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,5470,190752,240,144,144,144,144,240,144,144,336,240,0,0,US-SRM
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,7931,233,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,16160,230928,2208,2208,2208,2208,2208,2208,2208,2304,5568,2208,0,0,US-Ton
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,36398,1505,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,11529,245712,4656,4656,4656,4656,4656,4656,4656,4656,6960,4656,0,0,US-Var
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,886,997,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,5336,186768,480,480,480,480,480,480,480,480,576,480,0,0,US-Wkg
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,25511,21707,11463,11463,0,0,0,0,0,0,0,0,0,0,0,15552,0,15552,0,15552,0,15552,0,0,0,0,0,0,0,13965,213558,3072,3072,3072,3072,3072,3072,3072,3360,6768,3072,0,0,US-ARM
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,6189,3729,122879,122879,0,103909,103909,0,103909,103909,0,0,0,0,16896,0,16896,0,16896,0,16896,0,0,0,0,0,0,0,0,20105,230688,16704,16704,16704,16704,16704,16704,16704,17328,24384,16752,0,0,US-Me2
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,11331,29484,191904,191904,0,191904,191904,0,191904,191904,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,6709,191904,54624,54576,54576,54576,54576,54624,54576,56400,60720,54576,0,0,US-UMB
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,50346,13861,174528,174528,0,174528,174528,0,174528,174528,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,7628,174528,10992,10944,10944,10944,10896,10992,10944,12240,17664,10896,0,0,US-Vcp


In [ ]:
plot_df = na_df.loc[:, (na_df.sum(axis=0) != 0)]
plot_df

,TA_ERA,SW_IN_ERA,LW_IN_ERA,VPD_ERA,P_F,P_F_QC,P_ERA,PA_F,PA_F_QC,PA_ERA,NETRAD,PPFD_IN,G_F_MDS,G_F_MDS_QC,LE_F_MDS_QC,LE_CORR,H_F_MDS_QC,H_CORR,GPP_NT_VUT_REF,GPP_DT_VUT_REF,GPP_NT_CUT_REF,GPP_DT_CUT_REF,RECO_NT_VUT_REF,RECO_DT_VUT_REF,RECO_NT_CUT_REF,RECO_DT_CUT_REF,NEE_VUT_REF_qa,SW_DIF,EVI,NDVI,NIRv,b1,b2,b3,b4,b5,b6,b7,site_id
0,0,0,0,0,0,0,0,0,0,0,4325,22234,102266,102266,0,50149,0,50149,0,0,0,0,0,0,0,0,12287,245760,9696,9552,9552,9552,9552,9696,9552,10656,11280,9648,FR-Pue
0,0,0,0,0,0,0,0,0,0,0,1794,1483,0,0,0,0,0,0,0,0,0,0,0,0,0,0,11177,270768,57984,57792,57792,57792,57792,57984,57888,58416,77424,57792,US-NR1
0,0,0,0,0,0,0,0,0,0,0,2643,757,0,0,0,0,0,0,0,0,0,0,0,0,0,0,5470,190752,240,144,144,144,144,240,144,144,336,240,US-SRM
0,0,0,0,0,0,0,0,0,0,0,7931,233,0,0,0,0,0,0,0,0,0,0,0,0,0,0,16160,230928,2208,2208,2208,2208,2208,2208,2208,2304,5568,2208,US-Ton
0,0,0,0,0,0,0,0,0,0,0,36398,1505,0,0,0,0,0,0,0,0,0,0,0,0,0,0,11529,245712,4656,4656,4656,4656,4656,4656,4656,4656,6960,4656,US-Var
0,0,0,0,0,0,0,0,0,0,0,886,997,0,0,0,0,0,0,0,0,0,0,0,0,0,0,5336,186768,480,480,480,480,480,480,480,480,576,480,US-Wkg
0,0,0,0,0,0,0,0,0,0,0,25511,21707,11463,11463,0,0,0,0,0,15552,0,15552,0,15552,0,15552,13965,213558,3072,3072,3072,3072,3072,3072,3072,3360,6768,3072,US-ARM
0,0,0,0,0,0,0,0,0,0,0,6189,3729,122879,122879,103909,103909,103909,103909,16896,0,16896,0,16896,0,16896,0,20105,230688,16704,16704,16704,16704,16704,16704,16704,17328,24384,16752,US-Me2
0,0,0,0,0,0,0,0,0,0,0,11331,29484,191904,191904,191904,191904,191904,191904,0,0,0,0,0,0,0,0,6709,191904,54624,54576,54576,54576,54576,54624,54576,56400,60720,54576,US-UMB
0,0,0,0,0,0,0,0,0,0,0,50346,13861,174528,174528,174528,174528,174528,174528,0,0,0,0,0,0,0,0,7628,174528,10992,10944,10944,10944,10896,10992,10944,12240,17664,10896,US-Vcp


In [ ]:
site_df = plot_df.loc[plot_df['site_id'] == 'US-Vcp', :].drop(['site_id'], axis=1).T
site_df.reset_index(inplace=True)
site_df.rename(columns={"index": "data_type", 0:"count"}, inplace=True)
site_df

,data_type,count
0,TA_ERA,0
1,SW_IN_ERA,0
2,LW_IN_ERA,0
3,VPD_ERA,0
4,P_F,0
5,P_F_QC,0
6,P_ERA,0
7,PA_F,0
8,PA_F_QC,0
9,PA_ERA,0


In [ ]:
# Plot

plot_df = na_df.loc[:, (na_df.sum(axis=0) != 0)]
groups = plot_df.columns[:-1]
index = target_sites
total_offset = len(index) + 1
fig_data = []

subgroup_text = []

# Add the traces
for i, s in enumerate(index):
    site_df = plot_df.loc[plot_df['site_id'] == s, :].drop(['site_id'], axis=1).T
    if site_df.shape[1] == 0:
        continue
    site_df.reset_index(inplace=True)
    site_df.rename(columns={"index": "data_type", 0:"count"}, inplace=True)
    
    bar_plot = go.Bar(
        x = site_df["data_type"], y = site_df["count"],
        name = s,
        offsetgroup = i,
        text = s,
        hovertemplate = "%{x}<br>%{y}",
        showlegend = False,
    )
 
    fig_data.append(bar_plot)

fig = go.Figure(
    data=fig_data,
    layout=go.Layout(
        height = 600,
        width = 1400,
        legend_title_text= "QC Flag",
        title="Half Hourly Data NA Distributions",
        xaxis_title="Data Type",
        yaxis_title="Record Count",
        template='plotly_white'
    )  
)

fig.show()

# Export Fig
fig.write_html(data_dir + "\\figures\\GoldSitesNADistribution.html")